In [18]:
import kagglehub
import os
from lxml import etree
import pandas as pd
import json


In [19]:
path = kagglehub.dataset_download("stanislavlevendeev/hazmat-detection")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\stani\.cache\kagglehub\datasets\stanislavlevendeev\hazmat-detection\versions\2


In [20]:
# Load and parse the XML file
xml_file = path + "\cvat-for-video-dataset-hazmat-codes\\annotations.xml"
print("Path to xml file:", xml_file)
tree = etree.parse(xml_file)
# Get the root element
root = tree.getroot()
print("Root element:", root.tag)


Path to xml file: C:\Users\stani\.cache\kagglehub\datasets\stanislavlevendeev\hazmat-detection\versions\2\cvat-for-video-dataset-hazmat-codes\annotations.xml
Root element: annotations


<>:2: SyntaxWarning: invalid escape sequence '\c'
<>:2: SyntaxWarning: invalid escape sequence '\c'
C:\Users\stani\AppData\Local\Temp\ipykernel_23880\1816753982.py:2: SyntaxWarning: invalid escape sequence '\c'
  xml_file = path + "\cvat-for-video-dataset-hazmat-codes\\annotations.xml"


In [ ]:
tasks = root.find(".//tasks") 
tasks_json = {}
if tasks is not None:
    number_frames = 0
    for task in tasks.findall("task"):
        # Extract the id and name of each task
        task_id = task.find("id").text if task.find("id") is not None else "No ID"
        task_name = task.find("name").text if task.find("name") is not None else "No Name"
        task_source = task.find("source").text if task.find("source") is not None else "No source"
        task_frames = task.find("stop_frame").text if task.find("stop_frame") is not None else "No frames"
        if(tasks_json.get(task_id) is None):
            tasks_json[task_id] = {
                "id": task_id,
                "name": task_name,
                "source": task_source,
                "frames": task_frames,
                "frames_id": number_frames,
                "labels": []
            }
        number_frames += int(task_frames)
        print(f"Task ID: {task_id}, Task Name: {task_name}, Task Source: {task_source}")
else:
    print("No tasks found in the XML.")


Task ID: 138, Task Name: Task1, Task Source: 1690279852.mp4
Task ID: 142, Task Name: {task2, task3}, Task Source: 1690281365.mp4
Task ID: 143, Task Name: {task2, task3}, Task Source: 1690279852.mp4
Task ID: 148, Task Name: {task4, task5, task6, task7, task8, task9}, Task Source: 1692830440.mp4
Task ID: 149, Task Name: task10, Task Source: 1690801380.mp4
Task ID: 150, Task Name: task11, Task Source: 1691487366.mp4
Task ID: 151, Task Name: task12, Task Source: 1692787289.mp4
Task ID: 155, Task Name: task16, Task Source: 1692875102.mp4
Task ID: 158, Task Name: task19, Task Source: 1692945482.mp4
Task ID: 161, Task Name: task22, Task Source: 1693811855.mp4
Task ID: 162, Task Name: Task23, Task Source: 1693954819.mp4
Task ID: 163, Task Name: Task24, Task Source: 1693805101.mp4
Task ID: 164, Task Name: Task26, Task Source: 1693820172.mp4
Task ID: 166, Task Name: Task28, Task Source: 1692787289.mp4
Task ID: 167, Task Name: Task29, Task Source: 1691496786.mp4
Task ID: 168, Task Name: Task30, T

In [22]:
print(tasks_json)

{'138': {'id': '138', 'name': 'Task1', 'source': '1690279852.mp4', 'frames': '729', 'frames_id': 0, 'labels': []}, '142': {'id': '142', 'name': '{task2, task3}', 'source': '1690281365.mp4', 'frames': '732', 'frames_id': 729, 'labels': []}, '143': {'id': '143', 'name': '{task2, task3}', 'source': '1690279852.mp4', 'frames': '729', 'frames_id': 1461, 'labels': []}, '148': {'id': '148', 'name': '{task4, task5, task6, task7, task8, task9}', 'source': '1692830440.mp4', 'frames': '4964', 'frames_id': 2190, 'labels': []}, '149': {'id': '149', 'name': 'task10', 'source': '1690801380.mp4', 'frames': '734', 'frames_id': 7154, 'labels': []}, '150': {'id': '150', 'name': 'task11', 'source': '1691487366.mp4', 'frames': '1268', 'frames_id': 7888, 'labels': []}, '151': {'id': '151', 'name': 'task12', 'source': '1692787289.mp4', 'frames': '4908', 'frames_id': 9156, 'labels': []}, '155': {'id': '155', 'name': 'task16', 'source': '1692875102.mp4', 'frames': '996', 'frames_id': 14064, 'labels': []}, '158

In [ ]:
tracks = root.findall("track")
for track in tracks:
    attributes = track.attrib
    task_id = attributes.get("task_id")
    print("Attributes:", attributes)
    print("Task ID:", task_id)

    boxes = track.findall("box")
    for box in boxes:
        box_attributes = box.attrib
        lable_obj = {
            "absolute_frame": box_attributes.get("frame"),
            "relative_frame": int(box_attributes.get("frame")) - tasks_json[task_id]["frames_id"],
            "xtl": box_attributes.get("xtl"),
            "ytl": box_attributes.get("ytl"),
            "xbr": box_attributes.get("xbr"),
            "ybr": box_attributes.get("ybr")
        }
        for attr in box.findall("attribute"):
            lable_obj[attr.attrib.get("name")] = attr.text
        tasks_json[task_id]["labels"].append(lable_obj)


Attributes: {'id': '0', 'label': 'hazmat code', 'source': 'manual', 'task_id': '138', 'subset': 'default'}
Task ID: 138
Attributes: {'id': '1', 'label': 'hazmat code', 'source': 'manual', 'task_id': '138', 'subset': 'default'}
Task ID: 138
Attributes: {'id': '2', 'label': 'hazmat code', 'source': 'manual', 'task_id': '138', 'subset': 'default'}
Task ID: 138
Attributes: {'id': '3', 'label': 'hazmat code', 'source': 'manual', 'task_id': '138', 'subset': 'default'}
Task ID: 138
Attributes: {'id': '4', 'label': 'hazmat code', 'source': 'manual', 'task_id': '138', 'subset': 'default'}
Task ID: 138
Attributes: {'id': '5', 'label': 'hazmat code', 'source': 'manual', 'task_id': '138', 'subset': 'default'}
Task ID: 138
Attributes: {'id': '6', 'label': 'hazmat code', 'source': 'manual', 'task_id': '138', 'subset': 'default'}
Task ID: 138
Attributes: {'id': '7', 'label': 'hazmat code', 'source': 'manual', 'task_id': '142', 'subset': 'default'}
Task ID: 142
Attributes: {'id': '8', 'label': 'hazmat

In [24]:
rows = []

for task_id, task_data in tasks_json.items():
    for label in task_data["labels"]:
        # Combine task and label data into a single row
        row = {
            "Task ID": task_data["id"],
            "Task Name": task_data["name"],
            "Source": task_data["source"],
            "Frames": task_data["frames"],
            "Absolute Frame": label["absolute_frame"],
            "Relative Frame": label["relative_frame"],
            "XTL": label["xtl"],
            "YTL": label["ytl"],
            "XBR": label["xbr"],
            "YBR": label["ybr"],
            "Code": label["code"],
            "Issue": label["issue"]
        }
        rows.append(row)

# Create a DataFrame
df = pd.DataFrame(rows)

# Display the DataFrame
print(df.head())

# Save the DataFrame to a CSV file
output_file = "./data/labels_dataframe.csv"
df.to_csv(output_file, index=False)
print(f"CSV file '{output_file}' created successfully.")

  Task ID Task Name          Source Frames Absolute Frame  Relative Frame  \
0     138     Task1  1690279852.mp4    729             54              54   
1     138     Task1  1690279852.mp4    729             55              55   
2     138     Task1  1690279852.mp4    729             56              56   
3     138     Task1  1690279852.mp4    729             57              57   
4     138     Task1  1690279852.mp4    729             58              58   

      XTL     YTL     XBR     YBR     Code Issue  
0   29.87  506.88  190.69  554.96  83/2789  None  
1   65.26  504.87  225.50  552.95  83/2789  None  
2  131.98  503.67  291.63  551.76  83/2789  None  
3  198.69  502.48  357.76  550.57  83/2789  None  
4  241.62  498.68  400.10  546.77  83/2789  None  
CSV file './data/labels_dataframe.csv' created successfully.
